In [1]:
# Import general purpose packages
import numpy as np 
import pandas as pd 
import sys
import json
import os
import re

# This is what we are using for data preparation and ML part (thanks, Rafal, for great tutorial)
from sklearn.preprocessing import LabelBinarizer
from nltk.stem import WordNetLemmatizer

In [2]:
artists = pd.read_csv('artists-data.csv')
songs = pd.read_csv('lyrics-data.csv') #load the list of songs

In [3]:
data = pd.DataFrame() 
G=artists.Genre.unique()
for genre in G:
    Genre_artists = artists[artists['Genre']==genre] # filter artists by genre
    Genre_songs = pd.merge(songs, Genre_artists, how='inner', left_on='ALink', right_on='Link') #inner join of pop artists with songs to get only songs by pop artists
    Genre_songs = Genre_songs[['Genre', 'Artist', 'SName', 'Lyric','Idiom']].rename(columns={'SName':'Song'})#leave only columns of interest and rename some of them.
    Genre_songs = Genre_songs.dropna() # Remove incomplete records, cleanse lyrics
    #Genre_songs = Genre_songs[songs['Lyric']!='Instrumental'].head(SONGS_PER_GENRE).applymap(cleanse) #Remove instrumental compositions  and limit the size of final dataset
    data=pd.concat([data, Genre_songs])
    
    
data=data.loc[data['Idiom'] == 'ENGLISH']

data.index = range(len(data))


In [4]:
#Keep only 3 most frequent genrres
data=data.loc[data['Genre'].isin(['Rock','Pop','Hip Hop'])]
data.index = range(len(data))


In [5]:
#Convert Categorical genre to binary label
GenreBinarizer = LabelBinarizer().fit(data['Genre'])
Genre_Label = GenreBinarizer.transform(data['Genre'])


Genre_Label_df = pd.DataFrame(Genre_Label, columns =['G_H', 'G_P', 'G_R'])

final_data=pd.concat([data, Genre_Label_df], axis=1)

##### Shuffle data
final_data=final_data.sample(frac=1) 
final_data.index = range(len(final_data))
final_data=final_data.drop(columns=['Genre', 'Artist','Idiom'])

In [6]:
#Creat train,validation and test data
train_data=final_data[:87000]
validation_data=final_data[87000:111600]
test_data=final_data[111600:]

In [7]:
documents = []
stemmer = WordNetLemmatizer()
for sen in range(0, len(train_data)):
    temp=str(train_data['Lyric'].values[sen])
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(temp))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    

    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
    
train_Lyric=documents

In [8]:
documents = []
stemmer = WordNetLemmatizer()
for sen in range(0, len(validation_data)):
    temp=str(validation_data['Lyric'].values[sen])
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(temp))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    

    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
    
validation_Lyric=documents

In [9]:
documents = []
stemmer = WordNetLemmatizer()
for sen in range(0, len(test_data)):
    temp=str(test_data['Lyric'].values[sen])
    # Remove all the special characters
    document = re.sub(r'\W', ' ', str(temp))
    
    # remove all single characters
    document = re.sub(r'\s+[a-zA-Z]\s+', ' ', document)
    

    
    # Substituting multiple spaces with single space
    document = re.sub(r'\s+', ' ', document, flags=re.I)
    
    # Removing prefixed 'b'
    document = re.sub(r'^b\s+', '', document)
    
    # Converting to Lowercase
    document = document.lower()
    
    # Lemmatization
    document = document.split()

    document = [stemmer.lemmatize(word) for word in document]
    document = ' '.join(document)
    
    documents.append(document)
    
    
test_Lyric=documents

In [10]:
y_train=train_data[['G_H','G_P','G_R']].to_numpy()
y_validation=validation_data[['G_H','G_P','G_R']].to_numpy()
y_test=test_data[['G_H','G_P','G_R']].to_numpy()